In [52]:
import json
import pickle
from itertools import islice

from champs import top_champs

# Open the pkl file in read binary mode
with open('full_result.pickle', 'rb') as f:
    data = pickle.load(f)

In [53]:
my_champ = "Garen"

In [54]:
data[my_champ]

{'Gragas': '0.0793',
 "K'Sante": '0.0602',
 'Gwen': '0.0434',
 'Yone': '0.0345',
 'Jax': '0.0299',
 'Shyvana': '0.0234',
 'Pantheon': '0.0224',
 'Nasus': '0.0216',
 'Warwick': '0.0216',
 'Riven': '0.0209',
 'Tryndamere': '0.0195',
 'Ornn': '0.0195',
 'Irelia': '0.0187',
 'Renekton': '0.0128',
 'Singed': '0.0125',
 'Illaoi': '0.0108',
 'Kennen': '0.0102',
 'Akshan': '-0.0728',
 'Kayle': '-0.0539',
 'Camille': '-0.051',
 'Trundle': '-0.0489',
 'Quinn': '-0.0375',
 'Malphite': '-0.0366',
 'Tahm Kench': '-0.0351',
 'Akali': '-0.0324',
 'Rumble': '-0.0269',
 'Mordekaiser': '-0.0172',
 'Urgot': '-0.0157',
 'Sett': '-0.0153',
 'Fiora': '-0.0144',
 'Aatrox': '-0.0139',
 'Darius': '-0.0139',
 'Olaf': '-0.0135',
 'Gnar': '-0.0128',
 'Shen': '-0.0105'}

In [55]:
results = {}
for c in data:
    garen_data = data[my_champ].copy()
    for k, v in garen_data.items():
        if float(v) < 0:
            garen_data[k] = 0 if float(data[c].get(k, 0)) > 0 else float(garen_data[k])
            # if float(data[c].get(k, 0)) > 0:
            #     print(f"{c} counters weakness {k}")

    results[c] = 0
    for value in garen_data.values():
        if float(value) < 0:
            results[c] += 1

In [56]:
results = dict(sorted(results.items(), key=lambda item: item[1]))

for c in results:
    print(f"{c}: {results[c]}")

Malphite: 8
Pantheon: 9
Dr. Mundo: 10
Irelia: 10
Illaoi: 10
Nasus: 10
Ornn: 11
Riven: 11
Sion: 11
Teemo: 11
Aatrox: 12
Gangplank: 12
Jayce: 12
K'Sante: 12
Mordekaiser: 12
Sett: 12
Yorick: 12
Camille: 13
Jax: 13
Kled: 13
Quinn: 13
Renekton: 13
Tahm Kench: 13
Tryndamere: 13
Volibear: 13
Darius: 14
Gnar: 14
Shen: 14
Warwick: 14
Fiora: 15
Poppy: 15
Akali: 16
Cho'Gath: 16
Gragas: 16
Kennen: 16
Singed: 16
Urgot: 16
Vladimir: 16
Trundle: 17
Wukong: 17
Yasuo: 17
Garen: 18
Maokai: 18
Rengar: 18
Zac: 18


In [57]:
def weak_matrix(champ):
    results = {}
    for c in data:
        garen_data = data[champ].copy()
        for k, v in garen_data.items():
            if float(v) < 0:
                garen_data[k] = 0 if float(data[c].get(k, 0)) > 0 else float(garen_data[k])
                # if float(data[c].get(k, 0)) > 0:
                #     print(f"{c} counters weakness {k}")
    
        results[c] = 0
        for value in garen_data.values():
            if float(value) < 0:
                results[c] += 1
        results = dict(sorted(results.items(), key=lambda item: item[1]))
    return dict(islice(results.items(), 7))

In [58]:
def score(champ1, champ2):
    results = {}
    garen_data = data[champ1].copy()
    for k, v in garen_data.items():
        if float(v) < 0:
            garen_data[k] = 0 if float(data[champ2].get(k, 0)) > 0 else float(garen_data[k])
            # if float(data[c].get(k, 0)) > 0:
            #     print(f"{c} counters weakness {k}")

    results[champ2] = 0
    for value in garen_data.values():
        if float(value) < 0:
            results[champ2] += 1
    results = dict(sorted(results.items(), key=lambda item: item[1]))
    return dict(islice(results.items(), 7))

In [59]:
wm = weak_matrix(my_champ)
wm

{'Malphite': 8,
 'Pantheon': 9,
 'Dr. Mundo': 10,
 'Irelia': 10,
 'Illaoi': 10,
 'Nasus': 10,
 'Ornn': 11}

In [60]:
def get_best_third_champ(champ):
    results = {}
    for c in weak_matrix(champ):
        batch = {}
        for c2 in weak_matrix(c):
            m = score(champ, c2)
            for k, v in m.items():
                batch[k] = v
        results[c] = batch
    return results

In [61]:
def evaluate_triplets(results, champ):
    final = {}
    for c in results:
        sums = {}
        for c2 in results[c]:
            sums[c2] = results[c][c2] + wm.get(c, 100)
            
        min_value = min(sums.values())
        min_keys = [key for key, value in sums.items() if value == min_value]
    
        final[c] = {}
        for k in min_keys:
            final[c][k] = sums[k]

    print(f"These are your best 3-trick combos for {champ}:\n")
    for c in final:
        for c2 in final[c]:
            print(f"- {champ}, {c}, {c2} (Score: {final[c][c2]})")
    print("\n* Score stands for amount of counters a combo has. Lower is better.")

In [62]:
evaluate_triplets(get_best_third_champ(my_champ), my_champ)

These are your best 3-trick combos for Garen:

- Garen, Malphite, Illaoi (Score: 18)
- Garen, Pantheon, Irelia (Score: 19)
- Garen, Pantheon, Dr. Mundo (Score: 19)
- Garen, Pantheon, Nasus (Score: 19)
- Garen, Dr. Mundo, Malphite (Score: 18)
- Garen, Irelia, Malphite (Score: 18)
- Garen, Illaoi, Malphite (Score: 18)
- Garen, Nasus, Malphite (Score: 18)
- Garen, Ornn, Malphite (Score: 19)

* Score stands for amount of counters a combo has. Lower is better.


In [63]:
# for c in top_champs:
#     evaluate_triplets(get_best_third_champ(c), c)